### Importar bibliotecas

In [25]:
from Grafo import *
import numpy as np
# import o alfabeto em codigo Ascii 
import string 
# manipular os dados de excel com xlrd
import xlrd

## Alfabeto Σ e Estados


In [10]:
tamanhoAlfabeto = len(string.printable)
alfabeto = []

def criaAlfabeto():#funcao para criar o alfabeto
    alfabeto.extend(string.printable)  #cria uma nova lista contendo o alfabeto

criaAlfabeto()
#print(len(alfabeto))
#print (alfabeto)

# Ajuste do numero de estados para criar o automato
numEstados = 41 #son 40 estados en total pero como el rango de [0;40] tiene que se x<41
estadosList = [i for i in range(numEstados)]

estadosFinais=[1,3,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,38,39,40]

In [12]:
try:
    # import banco de dados palabras reservadas (excel.xls) 
    #excel = xlrd.open_workbook('C:/Users/gaboh/Documents/GitHub/BNF/Tabela-Simbolos2.xls') #import xlrd
    excel = xlrd.open_workbook('/home/moskbr/Documentos/git/BNF/Tabela-Simbolos2.xls')
    sh = excel.sheet_by_index(0) #Folha 1 ou sheet 1
except:
    print("Exception ->Tabela de simbolos tem que ser um doc Excel do tipo .xls")

tabelaSimbolos ={} #dictionary
numColumnas = sh.nrows-2
for key in range(numColumnas):
    cellCadeia = sh.cell(key+1,1).value
    cellCategoria = sh.cell(key+1,2).value
    tabelaSimbolos[cellCadeia] = cellCategoria
# Tabela Simbolos -> colocar as pabras dentro de uma tabela hash ( Em python o dictionary representa a tabela hash )
print(tabelaSimbolos)
# busca na tabela de simbolos se existe a palavra 
tabelaSimbolos["While"]

{'While': 'TK_While', 'If': 'TK_If', 'Main': 'TK_Main', 'Fun': 'TK_Fun', 'Int': 'TK_Inteiro', 'Float': 'TK_Flutuante', 'Char': 'TK_Char', 'Write': 'TK_Write', 'Read': 'TK_Read', '\\n': 'TK_New_Line', '(': 'TK_Abre_Parenteses', ')': 'TK_Fecha_Parenteses', '{': 'TK_Abre_Chaves', '}': 'TK_Fecha_Chaves', '[': 'TK_Abre_Colchetes', ']': 'TK_Fecha_Colchetes', '=': 'TK_Atribui_Valor', '+': 'TK_Soma', '-': 'TK_Subtracao', '*': 'TK_Multiplicacao', '/': 'TK_Divisao', '%': 'TK_Mod', '+=': 'TK_Incrementa_Valor', '-=': 'TK_Decrementa_Valor', '*=': 'TK_Multiplica_Valor', '/=': 'TK_Divide_Valor', '%=': 'TK_Mod_Valor', '++': 'TK_Incrementa_Um', '--': 'TK_Decrementa_Um', '==': 'TK_Verifica_Igual', '!=': 'TK_Verifica_Diferente', '<>': 'TK_Verifica_Diferente', '>': 'TK_Maior', '<': 'TK_Menor', '>=': 'TK_Maior_Igual', '<=': 'TK_Menor_Igual', '!': 'TK_Nega', ';': 'TK_Final_Lexema', '"': 'TK_Aspa', '//': 'TK_Comentario', '&': 'TK_E_Comercial', ',': 'TK_Virgula', '&&': 'TK_And', '||': 'TK_Or', 'x': 'TK_Identi

'TK_While'

### Construcao do Automato 

In [30]:
from traceback import print_tb


def main():
    grafo = Grafo()
    for x in range(numEstados): agregar(grafo, estadosList[x]) # Cria o grafo de 40 estados
    letras=[]
    algarismos=[]

    for cont in range(0,52): #armazena so as letras do alfabeto
        letras.append(alfabeto[cont+10])
    for cont in range(0,10): # armazena so os numeros do alfabeto 
        algarismos.append(alfabeto[cont])

    #for cont in algarismos: #armazena em cada transicao um digito
    relacionar(grafo, estadosList[0], estadosList[1], algarismos)
    relacionar(grafo, estadosList[1], estadosList[1], algarismos)
    relacionar(grafo, estadosList[2], estadosList[3], algarismos)
    relacionar(grafo, estadosList[3], estadosList[3], algarismos)
    relacionar(grafo, estadosList[4], estadosList[4], algarismos)

    #for cont in letras: #armazena em cada transicao o conjunto de letras
    relacionar(grafo, estadosList[0], estadosList[4], letras)
    relacionar(grafo, estadosList[4], estadosList[4], alfabeto)
    
    relacionar(grafo, estadosList[1], estadosList[2], '.')
    relacionar(grafo, estadosList[4], estadosList[4], '_')
    relacionar(grafo, estadosList[0], estadosList[5], '\\')
    relacionar(grafo, estadosList[5], estadosList[6], 'n')
    relacionar(grafo, estadosList[5], estadosList[7], '\\')
    relacionar(grafo, estadosList[0], estadosList[8], '(')
    relacionar(grafo, estadosList[0], estadosList[9], ')')
    relacionar(grafo, estadosList[0], estadosList[10], '{')
    relacionar(grafo, estadosList[0], estadosList[11], '}')
    relacionar(grafo, estadosList[0], estadosList[12], '[')
    relacionar(grafo, estadosList[0], estadosList[13], ']')
    relacionar(grafo, estadosList[0], estadosList[14], '=')
    relacionar(grafo, estadosList[14], estadosList[15], '=')
    relacionar(grafo, estadosList[0], estadosList[16], '+')
    relacionar(grafo, estadosList[16], estadosList[17], '=')
    relacionar(grafo, estadosList[16], estadosList[18], '+')
    relacionar(grafo, estadosList[0], estadosList[19], '-')
    relacionar(grafo, estadosList[19], estadosList[20], '=')
    relacionar(grafo, estadosList[19], estadosList[21], '-')
    relacionar(grafo, estadosList[0], estadosList[22], '*')
    relacionar(grafo, estadosList[22], estadosList[23], '=')
    relacionar(grafo, estadosList[0], estadosList[24], '/')
    relacionar(grafo, estadosList[24], estadosList[25], '=')
    relacionar(grafo, estadosList[0], estadosList[26], '%')
    relacionar(grafo, estadosList[26], estadosList[27], '=')
    relacionar(grafo, estadosList[0], estadosList[28], '!')
    relacionar(grafo, estadosList[28], estadosList[29], '=')
    relacionar(grafo, estadosList[0], estadosList[30], '>')
    relacionar(grafo, estadosList[30], estadosList[31], '=')
    relacionar(grafo, estadosList[0], estadosList[32], '<')
    relacionar(grafo, estadosList[32], estadosList[33], '=')
    relacionar(grafo, estadosList[32], estadosList[29], '>')
    relacionar(grafo, estadosList[0], estadosList[34], '&')
    relacionar(grafo, estadosList[34], estadosList[35], '&')
    relacionar(grafo, estadosList[0], estadosList[36], '|')
    relacionar(grafo, estadosList[36], estadosList[37], '|')
    relacionar(grafo, estadosList[0], estadosList[38], ';')
    relacionar(grafo, estadosList[0], estadosList[39], '"')
    relacionar(grafo, estadosList[0], estadosList[40], ',')

    #imprimeGrafo(grafo)
    

    # verificação de palavra
    w = "int"
    print(verificaPalavra(grafo,w,estadosFinais))
            
    #def verificaPalvra(grafo, letra, totalPalavras, caso, num,  ultimaLetra,  estadoFinal):
   

# O automato vai retornar True se a palvra foi aceita e se verifica na tabela de simbolos 
# Se e False a Classe Erro dispara o tipo de erro 

### Classe Error

### Verifica a palavra na tabela de simbolos

[1, 'au']


In [31]:
if __name__ == "__main__":
    main()

NameError: name 'verificaPalavra' is not defined